In [2]:
# Data import
import pyomo.environ as pe
import math

# Data
factories = ["IJmuiden", "Segal", "South Wales"]
rebars = {"A": {"length": 2.4}, "B": {"length": 3.6}, "C": {"length": 4.2}}
longbars = {1: {"length": 9}, 2: {"length": 12}}
diameter = 0.057  # in meters
density = 7.85  # tons/m^3
production_capacity = {"IJmuiden": 12, "Segal": 10, "South Wales": 28}
customers = ["Bochum", "Boenen", "Dortmund", "Gelsenkirchen", "Hagen", "Iserlohn", "Neuss", "Schwerte"]
periods = [1, 2, 3, 4]

demand = {
    "A": {
        "Bochum": [2, 6, 5, 3],
        "Boenen": [4, 8, 5, 10],
        "Dortmund": [2, 7, 6, 5],
        "Gelsenkirchen": [5, 5, 5, 5],
        "Hagen": [19, 23, 25, 16],
        "Iserlohn": [13, 19, 17, 14],
        "Neuss": [20, 16, 14, 26],
        "Schwerte": [4, 5, 3, 4]
    },
    "B": {
        "Bochum": [4, 5, 7, 8],
        "Boenen": [5, 8, 12, 13],
        "Dortmund": [4, 5, 8, 10],
        "Gelsenkirchen": [9, 10, 6, 6],
        "Hagen": [15, 33, 31, 33],
        "Iserlohn": [22, 26, 20, 27],
        "Neuss": [12, 23, 30, 30],
        "Schwerte": [2, 8, 2, 6]
    },
    "C": {
        "Bochum": [6, 7, 7, 7],
        "Boenen": [6, 10, 15, 12],
        "Dortmund": [7, 6, 4, 12],
        "Gelsenkirchen": [10, 9, 9, 10],
        "Hagen": [12, 35, 33, 38],
        "Iserlohn": [14, 25, 23, 24],
        "Neuss": [22, 32, 31, 31],
        "Schwerte": [5, 6, 7, 2]
    }
}

fixed_cost = {"IJmuiden": 130, "Segal": 150, "South Wales": 100}
v = 0.5  # variable cost €/km/tonne
distance = {
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901}
}

# Precompute parameters
w_r = {r: rebars[r]["length"] for r in rebars}
w_l = {l: longbars[l]["length"] for l in longbars}
area = math.pi * (diameter / 2) ** 2


# possible number of long bars per long bar type per factory 
N = {
    1: {"IJmuiden": 67, "Segal": 56, "South Wales": 156},
    2: {"IJmuiden": 50, "Segal": 42, "South Wales": 117}
}

possible_longbars = {} 

for l in longbars:
    possible_longbars[l] = {} 
    for f in factories:
        possible_longbars[l][f] = list(range(1, N.get(l, {}).get(f, 0) + 1))

# Defining big M
# List to store total demand per customer
total_demand_per_customer = []

# Loop over each customer
for a in customers:
    total_demand = sum(demand[r][a][t-1] for r in rebars for t in periods)  # Sum over all rebars and periods
    total_demand_per_customer.append(total_demand)

# Get the maximum demand across all customers
M = max(total_demand_per_customer)
  

In [3]:
# New data for 2: Inventory capacities (from Table 7)
inventory_capacity = {
    "Bochum": 10,
    "Boenen": 7,
    "Dortmund": 12,
    "Gelsenkirchen": 10,
    "Hagen": 12,
    "Iserlohn": 9,
    "Neuss": 8,
    "Schwerte": 5
}

model2 = pe.ConcreteModel()

# Initialize sets for model 2
model2.F = pe.Set(initialize=factories)
model2.R = pe.Set(initialize=rebars.keys())
model2.L = pe.Set(initialize=longbars.keys())
model2.A = pe.Set(initialize=customers)
model2.T = pe.Set(initialize=periods)

valid_x_indices = []
for l in possible_longbars:
    for f in possible_longbars[l]:
        for n in possible_longbars[l][f]:  # Only valid values of n
            valid_x_indices.append((l, n, f))

# Create an indexed set for (L, F, N)
model2.L_N_F = pe.Set(dimen=3, initialize=valid_x_indices)

# Define variables
model2.x = pe.Var(model2.R, model2.L_N_F,  model2.T, domain=pe.NonNegativeIntegers)
model2.y = pe.Var(model2.L_N_F, model2.T, domain=pe.Binary)
model2.z = pe.Var(model2.F, model2.A, model2.T, domain=pe.Binary)

# Add new variables to model2
model2.i = pe.Var(model2.A, model2.R, model2.T, domain=pe.NonNegativeIntegers)
model2.q = pe.Var(model2.R, model2.F, model2.A, model2.T, domain=pe.NonNegativeIntegers)


# Constraints 
# 1. Cutting constraints - same as Q1
model2.cutting_cnstr = pe.ConstraintList()
for f in model2.F:
    for t in model2.T:
        for l in model2.L:
            for n in possible_longbars[l][f]:
                expression = sum(model2.x[r, l, n, f, t] * w_r[r] for r in model2.R) <= model2.y[l, n, f, t] * w_l[l]
                model2.cutting_cnstr.add(expression)

# 2. Production capacity - same as Q1
model2.capacity_cnstr = pe.ConstraintList()
for f in model2.F:
    for t in model2.T:
        expression = sum(model2.y[l, n, f, t] * w_l[l] for n in possible_longbars[l][f] for l in model2.L) * density * area <= production_capacity[f]
        model2.capacity_cnstr.add(expression)

# Factory output
model2.factory_output = pe.ConstraintList()
for f in model2.F:
    for r in model2.R:
        for t in model2.T:
            expression = sum(model2.q[(r, f, a, t)] for a in model2.A) == sum(model2.x[(r, l, n, f, t)] for n in possible_longbars[l][f] for l in model2.L)
            model2.factory_output.add(expression)

# Link z to q
model2.enforce_factory_usage = pe.ConstraintList()
for f in model2.F:
    for a in model2.A:
        for t in model2.T:
            expression = sum(model2.q[(r, f, a, t)] for r in model2.R) <= M * model2.z[f, a, t]
            model2.enforce_factory_usage.add(expression)

# Demand 
model2.demand = pe.ConstraintList()
for a in model2.A:
    for r in model2.R:
        for t in model2.T:
            shipped = sum(model2.q[(r, f, a, t)] for f in model2.F) 
            if t == 1:
                expression = shipped - model2.i[(a,r,t)] == demand[r][a][t-1] 
                model2.demand.add(expression)
            else:
                expression = shipped - model2.i[(a,r,t)] + model2.i[(a,r,t-1)] == demand[r][a][t-1] 
                model2.demand.add(expression)

# Single sourcing
model2.single_sourcing_cnstr = pe.ConstraintList()
for a in model2.A:
    for t in model2.T:
        expression = sum(model2.z[(f,a,t)] for f in model2.F) <= 1
        model2.single_sourcing_cnstr.add(expression)

# Inventory
model2.inventory = pe.ConstraintList()
for a in model2.A:
    for t in model2.T:
        expression = sum(model2.i[(a,r,t)]* w_r[r] for r in model2.R) * density * area <= inventory_capacity[a]
        model2.inventory.add(expression)

#  production triggers customer assignment constraint (link y to z)
model2.prod_triggers_customer_assignment = pe.ConstraintList()
for f in model2.F:
    for t in model2.T:
        # Sum over all r, l, n as specified
        expression = sum(model2.y[ l, n, f, t]
                         for l in model2.L 
                         for n in possible_longbars[l][f]) <= M * sum(model2.z[f, a, t] for a in model2.A)
        
        # Add the constraint to the model
        model2.prod_triggers_customer_assignment.add(expression)


# Objective function
objExpr = sum(model2.z[f, a, t] * (fixed_cost[f] +  v * distance[a][f] * sum(model2.q[(r, f, a, t)] * w_r[r] for r in model2.R) * density * area) for f in model2.F for a in model2.A for t in model2.T)  
model2.obj = pe.Objective(expr=objExpr, sense=pe.minimize)

# Solve the model
solver = pe.SolverFactory('gurobi')
result2 = solver.solve(model2, tee=True, options={'TimeLimit':3600})

# Display results
print(f"\nSolver Status: {result2.solver.status}")
print(f"Termination Condition: {result2.solver.termination_condition}")
if result2.solver.status == pe.SolverStatus.ok and result2.solver.termination_condition == pe.TerminationCondition.optimal:
    print(f"Objective value (Total Cost): {pe.value(model2.obj):.2f} €")
    print("\nFactory assignments (z[f,a,t] = 1):")
    for f in model2.F:
        for a in model2.A:
            for t in model2.T:
                if pe.value(model2.z[f, a, t]) > 0.5:
                    print(f"z[{f}, {a}, {t}] = 1")
    print("\nQuantities shipped (q[r,f,a,t] > 0):")
    for r in model2.R:
        for f in model2.F:
            for a in model2.A:
                for t in model2.T:
                    if pe.value(model2.q[r,f,a,t]) > 0:
                        print(f"q[{r}, {f}, {a}, {t}] = {pe.value(model2.q[r,f,a,t]):.0f}")
    print("\nRebars cut (x[r,l,n,f,t] > 0):")
    for r in model2.R:
        for l in model2.L:
            for f in model2.F:
                for n in possible_longbars[l][f]:
                    for t in model2.T:
                        if pe.value(model2.x[r, l, n, f, t]) > 0:
                            print(f"x[{r}, {l}, {n}, {f}, {t}] = {pe.value(model2.x[r, l, n, f, t]):.0f}")
    print("\nLong bars used (y[l,n,f,t] > 0):")
    for l in model2.L:
        for f in model2.F:
            for n in possible_longbars[l][f]:
                for t in model2.T:
                    if pe.value(model2.y[l, n, f, t]) > 0:
                        print(f"y[{l}, {n}, {f}, {t}] = {pe.value(model2.y[l, n, f, t]):.0f}")
    
else:
    print("No optimal solution found.")


Set parameter Username
Set parameter LicenseID to value 2618603
Academic license - for non-commercial use only - expires 2026-02-05
Read LP format model from file /var/folders/dh/_xb_27wd2qg4j5p7s0bnfz1w0000gn/T/tmpy5hc1xty.pyomo.lp
Reading time = 0.05 seconds
x1: 2268 rows, 8288 columns, 17864 nonzeros
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23H417)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  3600

Optimize a model with 2268 rows, 8288 columns and 17864 nonzeros
Model fingerprint: 0x9534496f
Model has 288 quadratic objective terms
Variable types: 0 continuous, 8288 integer (2048 binary)
Coefficient statistics:
  Matrix range     [5e-02, 3e+02]
  Objective range  [1e+02, 2e+02]
  QObjective range [7e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve remove

In [9]:
# Analysis of production and demand

total_demand_per_period = {1: 0, 2: 0, 3: 0, 4: 0}
for r in rebars:
    for customer in demand[r]:
        for t in range(4):  # For each period
            # Demand in tonnes for each rebar type, customer, and period
            demand_in_tonnes = demand[r][customer][t] * w_r[r] * density * area
            total_demand_per_period[t + 1] += demand_in_tonnes

# Initialize a dictionary to store production data
factory_production = {f: {t: 0 for t in range(1, 5)} for f in ["IJmuiden", "Segal", "South Wales"]}
total_production_per_period = {t: 0 for t in range(1, 5)}  # Total production for each period

# Extract production values from model2
for f in factory_production:
    for t in factory_production[f]:
        total_tonnes = sum(
            pe.value(model2.y[l, n, f, t]) * w_l[l] * density * area
            for l in model2.L for n in possible_longbars[l][f]
            if pe.value(model2.y[l, n, f, t]) > 0.5
        )
        factory_production[f][t] = round(total_tonnes, 2)  # Round to 2 decimals
        total_production_per_period[t] += total_tonnes  # Update total production per period

# Print table
print("Factory Production (Tonnes per Period)")
print("-" * 80)
print(f"{'Factory':<15} {'P1':<10} {'P2':<10} {'P3':<10} {'P4':<10} ")
print("-" * 80)

# Print the factory production values
for f, periods in factory_production.items():
    row = f"{f:<15}"
    for t in range(1, 5):
        row += f"{periods[t]:<10}"
    print(row)

# Print total production across all factories
row = f"{'Total Production':<15}"
for t in range(1, 5):
    row += f"{total_production_per_period[t]:<10}"
row += f"{round(sum(total_production_per_period.values()), 2):<15.2f}"  # Rounded total production
print(row)

# Print total demand per period
row = f"{'Total Demand':<15}"
for t in total_demand_per_period:
    row += f"{round(total_demand_per_period[t], 2):<10}"
print(row)

print()
# Cost per period

total_cost_per_period = {t: 0 for t in model2.T}  # Dictionary to store total cost for each period

# Loop through each period, customer, and factory to calculate the total cost per period
for t in model2.T:
    period_cost = 0  # Initialize the cost for the current period
    for f in model2.F:
        for a in model2.A:
            # Calculate the cost of transportation and fixed costs for each factory, customer, and period
            period_cost += pe.value(model2.z[f, a, t]) * (
                fixed_cost[f] + v * distance[a][f] * sum(demand[r][a][t-1] * w_r[r] for r in model2.R) * density * area
            )
    # Store the total cost for this period
    total_cost_per_period[t] = period_cost

# Print the total cost per period
for t in total_cost_per_period:
    print(f"Total cost for Period {t}: €{total_cost_per_period[t]:.2f}")


Factory Production (Tonnes per Period)
--------------------------------------------------------------------------------
Factory         P1         P2         P3         P4         
--------------------------------------------------------------------------------
IJmuiden       11.96     11.96     11.96     11.96     
Segal          9.98      9.98      9.98      9.98      
South Wales    0         0         0         0         
Total Production21.93427926856371421.93427926856371421.93427926856371421.93427926856371487.74          
Total Demand   15.48     23.73     23.06     25.02     

Total cost for Period 1: €2850.37
Total cost for Period 2: €2915.81
Total cost for Period 3: €2943.59
Total cost for Period 4: €3464.37


In [10]:
for f in factory_production:
    for t in factory_production[f]:
        total_tonnes = sum(
            pe.value(model2.y[l, n, f, t]) * w_l[l] * density * area
            for l in model2.L for n in possible_longbars[l][f]
            if pe.value(model2.y[l, n, f, t]) > 0.5
        )
        factory_production[f][t] = round(total_tonnes, 2)
        total_production_per_period[t] += total_tonnes  

        # Debugging prints
        print(f"Factory {f}, Period {t}: Total tonnes = {total_tonnes}")
        for l in model2.L:
            for n in possible_longbars[l][f]:
                y_value = pe.value(model2.y[l, n, f, t])
                if y_value > 0.5:
                    print(f"  y[{l}, {n}, {f}, {t}] = {y_value}")


Factory IJmuiden, Period 1: Total tonnes = 11.958689245052545
  y[1, 46, IJmuiden, 1] = 1.0
  y[2, 1, IJmuiden, 1] = 1.0
  y[2, 2, IJmuiden, 1] = 1.0
  y[2, 3, IJmuiden, 1] = 1.0
  y[2, 4, IJmuiden, 1] = 1.0
  y[2, 5, IJmuiden, 1] = 1.0
  y[2, 6, IJmuiden, 1] = 1.0
  y[2, 7, IJmuiden, 1] = 1.0
  y[2, 8, IJmuiden, 1] = 1.0
  y[2, 9, IJmuiden, 1] = 1.0
  y[2, 10, IJmuiden, 1] = 1.0
  y[2, 11, IJmuiden, 1] = 1.0
  y[2, 12, IJmuiden, 1] = 1.0
  y[2, 13, IJmuiden, 1] = 1.0
  y[2, 14, IJmuiden, 1] = 1.0
  y[2, 15, IJmuiden, 1] = 1.0
  y[2, 16, IJmuiden, 1] = 1.0
  y[2, 17, IJmuiden, 1] = 1.0
  y[2, 18, IJmuiden, 1] = 1.0
  y[2, 19, IJmuiden, 1] = 1.0
  y[2, 20, IJmuiden, 1] = 1.0
  y[2, 21, IJmuiden, 1] = 1.0
  y[2, 22, IJmuiden, 1] = 1.0
  y[2, 23, IJmuiden, 1] = 1.0
  y[2, 24, IJmuiden, 1] = 1.0
  y[2, 25, IJmuiden, 1] = 1.0
  y[2, 26, IJmuiden, 1] = 1.0
  y[2, 27, IJmuiden, 1] = 1.0
  y[2, 28, IJmuiden, 1] = 1.0
  y[2, 29, IJmuiden, 1] = 1.0
  y[2, 30, IJmuiden, 1] = 1.0
  y[2, 31, IJmuid

In [17]:
for f in factory_production:
    for t in factory_production[f]:
        for l in model2.L:
       
        
            total_longbars=0
            for n in possible_longbars[l][f]:
            
                total_longbars +=    pe.value(model2.y[l, n, f, t])
    
            # Debugging prints
            print(f"Factory {f}, Period {t}, longbar {l}  = {total_longbars}")
    

Factory IJmuiden, Period 1, longbar 1  = 1.0
Factory IJmuiden, Period 1, longbar 2  = 49.0
Factory IJmuiden, Period 2, longbar 1  = 1.0
Factory IJmuiden, Period 2, longbar 2  = 49.0
Factory IJmuiden, Period 3, longbar 1  = 5.0
Factory IJmuiden, Period 3, longbar 2  = 46.0
Factory IJmuiden, Period 4, longbar 1  = 9.0
Factory IJmuiden, Period 4, longbar 2  = 43.0
Factory Segal, Period 1, longbar 1  = 10.0
Factory Segal, Period 1, longbar 2  = 34.0
Factory Segal, Period 2, longbar 1  = 6.0
Factory Segal, Period 2, longbar 2  = 37.0
Factory Segal, Period 3, longbar 1  = 2.0
Factory Segal, Period 3, longbar 2  = 40.0
Factory Segal, Period 4, longbar 1  = 6.0
Factory Segal, Period 4, longbar 2  = 37.0
Factory South Wales, Period 1, longbar 1  = 0.0
Factory South Wales, Period 1, longbar 2  = 0.0
Factory South Wales, Period 2, longbar 1  = 0.0
Factory South Wales, Period 2, longbar 2  = 0.0
Factory South Wales, Period 3, longbar 1  = 0.0
Factory South Wales, Period 3, longbar 2  = 0.0
Factory

In [19]:
# Collect data into a dictionary
shipment_data = {}

for t in model2.T:
    for a in model2.A:
        for f in model2.F:
            for r in model2.R:
                shipped_qty = pe.value(model2.q[r, f, a, t]) if pe.value(model2.q[r, f, a, t]) > 0 else 0
                inventory_qty = pe.value(model2.i[a, r, t]) if pe.value(model2.i[a, r, t]) > 0 else 0

                if shipped_qty > 0 or inventory_qty > 0:
                    if (t, a) not in shipment_data:
                        shipment_data[(t, a)] = []
                    shipment_data[(t, a)].append((f, r, shipped_qty, inventory_qty))

# Print table
print(f"{'Period':<10}{'Customer':<15}{'Factory':<15}{'Rebar Type':<12}{'Shipped Qty':<15}{'Inventory':<15}")
print("=" * 80)

for (t, a), shipments in shipment_data.items():
    for f, r, shipped_qty, inventory_qty in shipments:
        print(f"{t:<10}{a:<15}{f:<15}{r:<12}{shipped_qty:<15}{inventory_qty:<15}")


Period    Customer       Factory        Rebar Type  Shipped Qty    Inventory      
1         Bochum         IJmuiden       A           8.0            6.0            
1         Bochum         IJmuiden       B           9.0            5.0            
1         Bochum         IJmuiden       C           13.0           7.0            
1         Bochum         Segal          A           0              6.0            
1         Bochum         Segal          B           0              5.0            
1         Bochum         Segal          C           0              7.0            
1         Bochum         South Wales    A           0              6.0            
1         Bochum         South Wales    B           0              5.0            
1         Bochum         South Wales    C           0              7.0            
1         Boenen         IJmuiden       A           0              8.0            
1         Boenen         IJmuiden       B           0              8.0            
1   

In [4]:
# Initialize dictionary to store the shipment and inventory data
shipment_data = {}

# Collect data from the model
for t in model2.T:
    for f in model2.F:
        for a in model2.A:
            for r in model2.R:
                # Get the shipped quantity and inventory
                shipped_qty = pe.value(model2.q[r, f, a, t]) if pe.value(model2.q[r, f, a, t]) > 0 else 0
                inventory_qty = pe.value(model2.i[a, r, t]) if pe.value(model2.i[a, r, t]) > 0 else 0

                # Only store non-zero shipments or inventory
                if shipped_qty > 0 or inventory_qty > 0:
                    if (t, f, r) not in shipment_data:
                        shipment_data[(t, f, r)] = {}

                    shipment_data[(t, f, r)][a] = (shipped_qty, inventory_qty)

# Print table header with customers as columns
header = f"{'Period':<10}{'Factory':<15}{'Rebar Type':<12}"
for customer in model2.A:
    header += f"{customer} Shipped Qty    {customer} Inventory    "

print(header)
print("=" * len(header))

# Print the data row by row for each period, factory, and rebar type
for (t, f, r), customer_data in shipment_data.items():
    # Start a new row with period, factory, and rebar type
    row = f"{t:<10}{f:<15}{r:<12}"

    # For each customer, append their respective shipped quantity and inventory
    for customer in model2.A:
        if customer in customer_data:
            shipped_qty, inventory_qty = customer_data[customer]
            row += f"{shipped_qty:<15} {inventory_qty:<15} "
        else:
            # If there's no data for this customer, just add zeros
            row += f"{0:<15} {0:<15} "

    print(row)


Period    Factory        Rebar Type  Bochum Shipped Qty    Bochum Inventory    Boenen Shipped Qty    Boenen Inventory    Dortmund Shipped Qty    Dortmund Inventory    Gelsenkirchen Shipped Qty    Gelsenkirchen Inventory    Hagen Shipped Qty    Hagen Inventory    Iserlohn Shipped Qty    Iserlohn Inventory    Neuss Shipped Qty    Neuss Inventory    Schwerte Shipped Qty    Schwerte Inventory    
1         IJmuiden       A           8.0             6.0             0               8.0             15.0            13.0            6.0             1.0             0               2.0             13.0            0               0               0               4.0             0               
1         IJmuiden       B           9.0             5.0             0               8.0             17.0            13.0            11.0            2.0             0               0               25.0            3.0             0               0               2.0             0               
1         IJmuid